---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


In [10]:
def date_sorter():
    
    main = pd.DataFrame(df)
    check = main.reset_index()
    check.columns = ['index1','text'] 
    
    # 12/05/2015 12-05-2015
    df1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})')
    df1 = df1.reset_index()
    df1.columns = ['index','match','month','day','year']
    dates1 = pd.DataFrame(df1)
    dates1['year'] = dates1['year'].astype('int64')
    for i in np.arange(len(dates1)):
        if dates1.iloc[i,4] < 100:
            dates1.iloc[i,4]= dates1.iloc[i,4] + 1900
    dates1['day'] = dates1['day'].astype('int64')
    dates1['month'] = dates1['month'].astype('int64')
    

    
    #function to convert Categorical month to Numeric by taking the dataframe, column value as parameters.

    def chmonth(df,col):
        month = []
        for index,row in df.iterrows():
            if row[col] == 'Jan' or row[col] == 'January':
                y = 1
            elif row[col] == 'Feb' or row[col] == 'February':
                y = 2
            elif row[col] == 'Mar' or row[col] == 'March':
                y = 3
            elif row[col] == 'Apr' or row[col] == 'April':
                y = 4
            elif row[col] == 'May':
                y = 5
            elif row[col] == 'June' or row[col] == 'Jun':
                y = 6
            elif row[col] == 'Jul' or row[col] == 'July':
                y = 7
            elif row[col] == 'Aug' or row[col] == 'August':
                y = 8    
            elif row[col] == 'Sep' or row[col] == 'September':
                y = 9
            elif row[col] == 'Oct' or row[col] == 'October':
                y = 10
            elif row[col] == 'Nov' or row[col] == 'November':
                y = 11
            elif row[col] == 'Dec' or row[col] == 'December':
                y = 12
            month.append(y)
        return month
    
        # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    df2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ]*)(?:(\d{1,2}[, -]*))(?:(\d{4}))')
    df2 = df2.reset_index()
    df2.columns = ['index','match','month1','day','year']
    df2['month1'] = df2['month1'].str.replace(".","")
    df2['day'] = df2['day'].str.replace(",","")
    df2['month1'] = df2['month1'].str.strip()
    df2['month1']  = df2['month1'].astype('str')
    
    dates2 = pd.DataFrame(df2)

    month = chmonth(dates2,2)
    mon_cat_num = pd.DataFrame(month,columns=['month'])
    dates2 = pd.concat([dates2,mon_cat_num],axis=1)
    dates2.drop(['month1'],axis=1,inplace=True)
    dates2['day'] = dates2['day'].astype('int64')
    dates2['year'] = dates2['year'].astype('int64')
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # Feb 2009; Sep 2009; Oct 2010
    df3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:,. -]* )(\d{4})')
    df3 = df3.reset_index()
    df3.columns = ['index','match','day','month2','year']
    df3['month2'] = df3['month2'].str.replace(",","")
    df3['month2'] = df3['month2'].str.strip()
    df3['month2']  = df3['month2'].astype('str')
    dates3 = pd.DataFrame(df3)
    dates3['day'] = dates3['day'].fillna(1)
    #dates3['day'] = dates3['day'].astype('int64')
    dates3['year'] = dates3['year'].astype('int64')
    month = chmonth(dates3,3)
    mon_cat_num1 = pd.DataFrame(month,columns=['month'])

    dates3 = pd.concat([dates3,mon_cat_num1],axis=1)
    dates3 = dates3.drop(['month2'],axis=1)
    

    
    # 6/2008; 12/2009
    df5 = df.str.extractall(r'[ -]+(\d{1,2})[/](\d{4})')
    df7 = df.str.extractall(r'(^\d\d?)[/](\d{4})')
    df8 = df.str.extractall(r'[\(](\d{1,2})[/](\d{4})[\)]')
    df10 = df.str.extractall(r'[a-z]+(\d{1,2})[/](\d{4})')
    df5 = df5.reset_index()
    df7 = df7.reset_index()
    df8 = df8.reset_index()
    df10 = df10.reset_index()
    df9 = pd.concat([df5,df7,df10],axis=0)
    
    
    df9.columns = ['index','match','month','year']

    df9['month'] = df9['month'].str.strip()

    dates5 = pd.DataFrame(df9)
    dates5['day']  = 1
    dates5['month'] = dates5['month'].astype('int64')
    dates5['year'] = dates5['year'].astype('int64')
    
    dates = pd.concat([dates1,dates2,dates3,dates5])
    
    dates_dup = dates.drop_duplicates('index')
    
    # Checking for dates like 2009,2000
    chk2 = list(dates_dup['index'])
    chk2.sort()
    chk1 = list(check['index1'])
    chk1.sort()
    ind = [x for x in chk1 if x not in chk2 ]
    
    last_part = []
    for index,row in check.iterrows():
        for i in ind:
            if row[0] == i:
                data = row
                last_part.append(data)
    last = pd.DataFrame(last_part)
    df6 = last['text'].str.extractall(r'(\d{4})')
    dates6 = pd.DataFrame(df6)
    dates6 = dates6.reset_index()
    dates6.columns = ['index','match','year']
    dates6['day'] = 1
    dates6['month'] = 1
    dates6['year'] = dates6['year'].astype('int64')
    
    dates_final = pd.concat([dates_dup,dates6])
    
    dates_final = dates_final.sort_values(['year','month','day'],ascending=[True,True,True])
    
    return dates_final['index']